In [14]:
import folium
import csv
import geopandas as gpd
import numpy as np
from shapely.geometry import Point, Polygon
from pyproj import Transformer

def create_hexagonal_grid_centers(country_boundary, radius):
    bbox = country_boundary.bounds
    min_x, min_y, max_x, max_y = bbox

    x_interval = radius * 3 / 2
    y_interval = np.sqrt(3) * radius

    grid_centers = []

    x_offset = 0
    y = min_y
    while y < max_y:
        x = min_x + x_offset
        while x < max_x:
            point = Point(x, y)
            if country_boundary.contains(point):
                grid_centers.append((x, y))
            x += x_interval
        x_offset = x_interval / 2 if x_offset == 0 else 0
        y += y_interval

    return grid_centers

def create_map(centers, radius):
    germany_center = (np.mean([47.270, 55.058]), np.mean([5.866, 15.042]))
    map_germany = folium.Map(location=germany_center, zoom_start=6)

    for center in centers:
        folium.Circle(
            location=(center[1], center[0]),  # Swap the order of latitude and longitude
            radius=radius * 1000,  # Convert radius to meters
            color='blue',
            fill=True,
            fill_color='blue',
            fill_opacity=0.15
        ).add_to(map_germany)

    return map_germany

def save_coordinates_to_csv(centers, filename):
    with open(filename, "w", newline="") as csvfile:
        csv_writer = csv.writer(csvfile)
        csv_writer.writerow(["Coordinates"])
        for center in centers:
            lat, lon = center
            csv_writer.writerow([f"{lon}, {lat}"])



if __name__ == "__main__":
    world = gpd.read_file(gpd.datasets.get_path('naturalearth_lowres'))
    germany_geo = world[world['name'] == 'Germany'].geometry.iloc[0]

    # Create a GeoDataFrame with Germany's geometry
    germany_gdf = gpd.GeoDataFrame({'geometry': [germany_geo]}, crs="EPSG:4326")

    # Project to UTM Zone 32N (EPSG:32632) which covers Germany
    germany_utm = germany_gdf.to_crs("EPSG:32632").geometry.iloc[0]
    radius = 50

    centers_utm = create_hexagonal_grid_centers(germany_utm, radius * 1000) # Convert radius to meters
    transformer = Transformer.from_crs("EPSG:32632", "EPSG:4326", always_xy=True)
    centers_geo = [transformer.transform(x, y) for x, y in centers_utm]


    map_germany = create_map(centers_geo, radius)
    map_germany.save("germany_circles_50km_range.html")
    save_coordinates_to_csv(centers_geo, "circle_coordinates_50km_range.csv")


In [ ]:
# spreadsheet url https://docs.google.com/spreadsheets/d/1hE23q-AiDKeRAHx3t8gDYHJJ9vH8HMcAlb5l5UCom1s/edit#gid=731999718